In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

from dpipe.io import load

In [2]:
path_base = Path('/gpfs/data/gpfs0/b.shirokikh/experiments/da/miccai2021_spottune/baseline/cc359_unet2d_one2all/')
path_oracle = Path('/gpfs/data/gpfs0/b.shirokikh/experiments/da/miccai2021_spottune/baseline/cc359_unet2d_oracle/')

meta = pd.read_csv('/gpfs/data/gpfs0/b.shirokikh/data/cc359/meta.csv', index_col='id')
meta.head()

,MRI,brain_mask,fold,tomograph_model,tesla_value,x,y,z
id,,,,,,,,
CC0030,images/CC0030_philips_15_42_F.nii.gz,masks/CC0030_philips_15_42_F_ss.nii.gz,4,philips,15,1.000008,0.888889,0.888889
CC0326,images/CC0326_ge_3_55_M.nii.gz,masks/CC0326_ge_3_55_M_ss.nii.gz,3,ge,3,1.000000,1.000000,1.000000
CC0187,images/CC0187_siemens_3_63_F.nii.gz,masks/CC0187_siemens_3_63_F_ss.nii.gz,1,siemens,3,1.329996,1.000000,1.000000
CC0114,images/CC0114_philips_3_60_M.nii.gz,masks/CC0114_philips_3_60_M_ss.nii.gz,5,philips,3,1.000000,1.000000,1.000000
CC0231,images/CC0231_siemens_3_56_M.nii.gz,masks/CC0231_siemens_3_56_M_ss.nii.gz,1,siemens,3,1.330005,1.000000,1.000000


In [3]:
records = []
for s in sorted(meta['fold'].unique()):
    res_row = {}
    
    # one2all results:
    sdices = load(path_base / f'experiment_{s}/test_metrics/sdice_score.json')
    for t in sorted(set(meta['fold'].unique()) - {s}):
        df_row = meta[meta['fold'] == t].iloc[0]
        target_name = df_row['tomograph_model'] + str(df_row['tesla_value'])
        
        ids_t = meta[meta['fold'] == t].index
        res_row[target_name] = np.mean([sdsc for _id, sdsc in sdices.items() if _id in ids_t])
    
    df_row = meta[meta['fold'] == s].iloc[0]
    source_name = df_row['tomograph_model'] + str(df_row['tesla_value'])
    sdices = {}
    for n_val in range(3):
        sdices = {**sdices,
                  **load(path_oracle / f'experiment_{s * 3 + n_val}/test_metrics/sdice_score.json')}
    res_row[source_name] = np.mean(list(sdices.values()))

    res_row[' '] = source_name
    records.append(res_row)

In [4]:
df = pd.DataFrame.from_records(records, index=' ')

In [5]:
df[df.index]

,siemens15,siemens3,ge15,ge3,philips15,philips3
,,,,,,
siemens15,0.858110,0.696067,0.364585,0.737310,0.581591,0.546969
siemens3,0.608170,0.872101,0.107161,0.658075,0.402445,0.371160
ge15,0.700031,0.629186,0.836391,0.580418,0.806356,0.625905
ge3,0.655559,0.731467,0.363212,0.886189,0.517381,0.359648
philips15,0.689121,0.645036,0.621807,0.589849,0.876841,0.532264
philips3,0.775209,0.717082,0.551598,0.744775,0.740148,0.855453
